In [ ]:
#Installing important libraries
!pip install fastapi uvicorn sentence-transformers faiss-cpu aiofiles

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.2 MB/s eta 0:00:00


In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 443.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.6/602.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 5.4 MB/s eta 0:00

In [ ]:
!pip install pypdf2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.4 MB/s eta 0:00:00


In [1]:
#Importing the Libraries
import os
import re
import json
from typing import List, Dict
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions
import numpy as np
from PyPDF2 import PdfReader

# Create HogwartsQ&A folder
# Create HogwartsQ&A folder
!mkdir -p "HogwartsQ&A"

# Change to the HogwartsQ&A directory
%cd "HogwartsQ&A"

ModuleNotFoundError: No module named 'fastapi'

In [ ]:
def extract_text_from_pdf(pdf_path: str) -> str:
    """
    Extracts text from a PDF file.

    Parameters:
    - pdf_path: The file path to the PDF document.

    Returns:
    - A string containing the extracted text from the PDF.
    """
    reader = PdfReader(pdf_path)  # Create a PdfReader object
    text = ""  # Initialize an empty string to hold the extracted text

    # Iterate through each page in the PDF
    for page in reader.pages:
        page_text = page.extract_text()  # Extract text from the current page
        if page_text:  # Check if any text was extracted
            text += page_text + "\n"  # Append the extracted text with a newline

    return text  # Return the complete extracted text

# Example usage
book_text = extract_text_from_pdf("/content/Harry Potter and the Prisoner of Azkaban.pdf")

In [ ]:
import re
from typing import List

def chunk_text(text: str, chunk_size: int = 150) -> List[str]:
    """
    Splits the input text into chunks of specified size.

    Parameters:
    - text: The input string to be chunked.
    - chunk_size: The maximum size of each chunk in characters (default is 150).

    Returns:
    - A list of text chunks.
    """
    chunks = []  # List to store the resulting text chunks
    current_chunk = []  # Temporary list to build the current chunk
    current_size = 0  # Variable to keep track of the current chunk size

    # Split the text into paragraphs based on double newlines
    for paragraph in re.split(r'\n\s*\n', text):
        words = paragraph.split()  # Split the paragraph into words
        for word in words:
            # Check if adding the new word exceeds the chunk size limit
            if current_size + len(word) > chunk_size and current_chunk:
                # If it exceeds, append the current chunk to chunks list
                chunks.append(' '.join(current_chunk))
                current_chunk = []  # Reset current chunk
                current_size = 0  # Reset current size

            # Append the word to the current chunk and update the size
            current_chunk.append(word)
            current_size += len(word) + 1  # +1 for the space after the word

    # Append any remaining words as the last chunk
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks  # Return the list of text chunks

# Example usage
chunks = chunk_text(book_text)

In [ ]:
# 3. Embedding Generation
model = SentenceTransformer('all-MiniLM-L6-v2')

# 4. Vector Database Integration
chroma_client = chromadb.Client()
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
collection = chroma_client.create_collection(name="hp_prisoner_of_azkaban", embedding_function=embedding_function)

# Add chunks to the collection
for i, chunk in enumerate(chunks):
    collection.add(
        documents=[chunk],
        metadatas=[{"source": f"chunk_{i}"}],
        ids=[f"id_{i}"]
    )

# 5. Query Handling & Retrieval
def get_relevant_chunks(query: str, n: int = 3) -> List[str]:
    results = collection.query(
        query_texts=[query],
        n_results=n
    )
    return results['documents'][0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# 6. Contextual Response Generation
def generate_response(query: str, relevant_chunks: List[str]) -> str:
    context = "\n".join(relevant_chunks)
    prompt = f"""
    Based on the following context from 'Harry Potter and the Prisoner of Azkaban', answer the question:

    Context:
    {context}

    Question: {query}

    Answer in the style of the Harry Potter universe, including relevant quotes if possible:
    """

    # For this example, we'll use a simple response generation.
    # In a production system, you would use a more sophisticated language model here.
    response = f"Here's what I found about '{query}':\n\n"
    response += "\n\n".join([f"• {chunk}" for chunk in relevant_chunks])
    return response

In [ ]:
from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse
import uvicorn
import subprocess
import time

app = FastAPI()


In [ ]:
!pip install pyngrok

In [ ]:
# 7. FastAPI Backend Integration

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List

# Initialize FastAPI app
app = FastAPI()

# CORS middleware for frontend to interact with backend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class QueryRequest(BaseModel):
    query: str

class QueryResponse(BaseModel):
    query: str
    answer: str
    relevant_chunks: List[str]

@app.post("/query", response_model=QueryResponse)
async def query_handler(request: QueryRequest):
    try:
        # 1. Get the query from the request
        query = request.query

        # 2. Retrieve relevant chunks
        relevant_chunks = get_relevant_chunks(query, n=3)  # Corrected here

        # 3. Generate the response based on relevant chunks
        response = generate_response(query, relevant_chunks)

        return QueryResponse(
            query=query,
            answer=response,
            relevant_chunks=relevant_chunks
        )
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [ ]:
!pip install fastapi[all] sentence-transformers chromadb PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 16.0 MB/s eta 0:00:00


In [ ]:
!ngrok authtoken 2n9QRdYrYamjHAS2VOSlnlv9yYN_4oKQidLzdLUVcB6d3aqsa


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install uvicorn

In [ ]:
!ngrok authtoken 2n9QRdYrYamjHAS2VOSlnlv9yYN_4oKQidLzdLUVcB6d3aqsa


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
app = FastAPI()

# CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Step 5: Define data models
class QueryRequest(BaseModel):
    query: str

class QueryResponse(BaseModel):
    query: str
    answer: str
    relevant_chunks: List[str]

In [ ]:
for i, chunk in enumerate(chunks):
    collection.add(
        documents=[chunk],
        metadatas=[{"source": f"chunk_{i}"}],
        ids=[f"id_{i}"]
    )

# Step 11: Function to get relevant chunks based on query
def get_relevant_chunks(query: str, n: int = 3) -> List[str]:
    results = collection.query(
        query_texts=[query],
        n_results=n
    )
    return results['documents'][0]

# Step 12: Generate response based on query
def generate_response(query: str, relevant_chunks: List[str]) -> str:
    context = "\n".join(relevant_chunks)
    prompt = f"""
    Based on the following context from 'Harry Potter and the Prisoner of Azkaban', answer the question:

    Context:
    {context}

    Question: {query}

    Answer in the style of the Harry Potter universe, including relevant quotes if possible:
    """

    response = f"Here's what I found about '{query}':\n\n"
    response += "\n\n".join([f"• {chunk}" for chunk in relevant_chunks])
    return response

Streaming output truncated to the last 5000 lines.


In [ ]:
@app.post("/query", response_model=QueryResponse)
async def query_handler(request: QueryRequest):
    try:
        query = request.query
        relevant_chunks = get_relevant_chunks(query, n=3)
        response = generate_response(query, relevant_chunks)

        return QueryResponse(
            query=query,
            answer=response,
            relevant_chunks=relevant_chunks
        )
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [ ]:
from pyngrok import ngrok


In [ ]:
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://c783-34-125-189-152.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
!pip install nest_asyncio


In [ ]:

@app.get("/items")
def read_items():
    return {"message": "This is your items endpoint"}

In [ ]:
import nest_asyncio
nest_asyncio.apply()

uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [405]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
/usr/lib/python3.10/selectors.py:191: RuntimeWarning: coroutine 'Server.serve' was never awaited
  return mapping[fileobj]
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [405]
